In [0]:
import os
import pathlib

from scipy import linalg
from skimage.io import imread
from torch.nn.functional import adaptive_avg_pool2d

try:
    from tqdm import tqdm
except ImportError:
    # If not tqdm is not available, provide a mock version of it
    def tqdm(x): return x

In [0]:
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets

from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

import time

import matplotlib.pyplot as plt

In [0]:
import numpy as np
import math

In [0]:
n_epochs = 200
# batch_size = 64
# changed to 50 since 10,000 test images and want to be divisible
batch_size = 50
lr = 0.0002
b1 = 0.5
b2 = 0.999
latent_dim = 1024
img_size = 32
channels = 1
sample_interval = 400

In [0]:
dims = 2048

In [0]:
from google.colab import drive
drive.mount('/content/drive')

BASE_DIR = ('drive/My Drive/LSGAN-Project')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
fashion_dataloader_test = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        BASE_DIR + '/data/fashion-mnist/test',
        train=False,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

In [0]:
def get_activations(all_images, model, batch_size=50, dims=2048,
                    cuda=False, verbose=False):
    """Calculates the activations of the pool_3 layer for all images.
    Params:
    -- model       : Instance of inception model
    -- batch_size  : Batch size of images for the model to process at once.
                     Make sure that the number of samples is a multiple of
                     the batch size, otherwise some samples are ignored. This
                     behavior is retained to match the original FID score
                     implementation.
    -- dims        : Dimensionality of features returned by Inception
    -- cuda        : If set to True, use GPU
    -- verbose     : If set to True and parameter out_step is given, the number
                     of calculated batches is reported.
    Returns:
    -- A numpy array of dimension (num images, dims) that contains the
       activations of the given tensor when feeding inception with the
       query tensor.
    """
    model.eval()

    if len(all_images) % batch_size != 0:
        print(('Warning: number of images is not a multiple of the '
               'batch size. Some samples are going to be ignored.'))
    if batch_size > len(all_images):
        print(('Warning: batch size is bigger than the data size. '
               'Setting batch size to data size'))
        batch_size = len(all_images)

    n_batches = len(all_images) // batch_size
    n_used_imgs = n_batches * batch_size

    pred_arr = np.empty((n_used_imgs, dims))

    for i in tqdm(range(n_batches)):
        if verbose:
            print('\rPropagating batch %d/%d' % (i + 1, n_batches),
                  end='', flush=True)
        start = i * batch_size
        end = start + batch_size

        images = all_images[start:end]

        batch = torch.from_numpy(images).type(torch.FloatTensor)
        if cuda:
            batch = batch.cuda()

        pred = model(batch)[0]

        # If model output is not scalar, apply global spatial average pooling.
        # This happens if you choose a dimensionality not equal 2048.
        if pred.shape[2] != 1 or pred.shape[3] != 1:
            pred = adaptive_avg_pool2d(pred, output_size=(1, 1))

        pred_arr[start:end] = pred.cpu().data.numpy().reshape(batch_size, -1)

    if verbose:
        print(' done')

    return pred_arr


def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).
    Stable version by Dougal J. Sutherland.
    Params:
    -- mu1   : Numpy array containing the activations of a layer of the
               inception net (like returned by the function 'get_predictions')
               for generated samples.
    -- mu2   : The sample mean over activations, precalculated on an
               representative data set.
    -- sigma1: The covariance matrix over activations for generated samples.
    -- sigma2: The covariance matrix over activations, precalculated on an
               representative data set.
    Returns:
    --   : The Frechet Distance.
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    # Product might be almost singular
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # Numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)


def calculate_activation_statistics(images, model, batch_size=50,
                                    dims=2048, cuda=False, verbose=False):
    """Calculation of the statistics used by the FID.
    Params:
    -- model       : Instance of inception model
    -- batch_size  : The images numpy array is split into batches with
                     batch size batch_size. A reasonable batch size
                     depends on the hardware.
    -- dims        : Dimensionality of features returned by Inception
    -- cuda        : If set to True, use GPU
    -- verbose     : If set to True and parameter out_step is given, the
                     number of calculated batches is reported.
    Returns:
    -- mu    : The mean over samples of the activations of the pool_3 layer of
               the inception model.
    -- sigma : The covariance matrix of the activations of the pool_3 layer of
               the inception model.
    """
    act = get_activations(images, model, batch_size, dims, cuda, verbose)
    mu = np.mean(act, axis=0)
    sigma = np.cov(act, rowvar=False)
    return mu, sigma


def _compute_statistics_of_path(images, model, batch_size, dims, cuda):
    m, s = calculate_activation_statistics(images, model, batch_size,
                                               dims, cuda)

    return m, s


def calculate_fid_given_paths(images, batch_size, cuda, dims):
    """Calculates the FID of two paths"""

    block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[dims]

    
    model = InceptionV3(output_blocks = [block_idx], normalize_input = False)
    if cuda:
        model.cuda()

    m1, s1 = _compute_statistics_of_path(images[0], model, batch_size,
                                         dims, cuda)
    m2, s2 = _compute_statistics_of_path(images[1], model, batch_size,
                                         dims, cuda)
    fid_value = calculate_frechet_distance(m1, s1, m2, s2)

    return fid_value

In [0]:
count_test = 0
for i, (imgs, _) in enumerate(fashion_dataloader_test):
    count_test+=imgs.shape[0]

In [0]:
count_test

10000

In [0]:
class Our_Generator(nn.Module):
    def __init__(self, batch_norm = True, momentum = 0.8):
        super(Our_Generator, self).__init__()
        
        self.init_size = img_size // 4

        def linear_block(in_feat, out_feat, normalize=batch_norm):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, momentum = momentum))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        # as specified in paper
        def deconv_block(input_channels, output_channels, kernel_size, stride, padding, output_padding, normalize=batch_norm):
            layers = [nn.ConvTranspose2d(input_channels, output_channels, kernel_size = kernel_size, stride = stride, padding = padding, output_padding = output_padding)]
            if normalize:
                layers.append(nn.BatchNorm2d(output_channels, momentum = momentum))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers


        self.first_layer = nn.Sequential(*linear_block(latent_dim, 128 *  self.init_size**2))

        # formula to keep 2D images the same size FOR CONVENIENCE
        # https://discuss.pytorch.org/t/how-to-keep-the-shape-of-input-and-output-same-when-dilation-conv/14338/2

        # I used this formula to keep the Image Sizes the same for deconvolutional layers
        # https://www.wolframalpha.com/input/?i=28+%3D+%2828+-+1%29+*+1+-2Z+%2B++P+%2B+%283-1%29+%2B+1

        # FORMULA FOR SIZE OF CONVOLUTIONAL AND DECONV LAYERS ARE DIFFERENT

        self.deconvolutional_part = nn.Sequential(
                *deconv_block(128, 128, kernel_size = 3, stride = 2, padding = 1, output_padding = 1),
                *deconv_block(128, 64, kernel_size = 3, stride = 2, padding = 1, output_padding = 1),
                # Taking really long to train
                # Very last deconv part, needs to be right size of image
                # channels is 1 for mnist, 3 for colored datasets
                # NO batch norm here, always
                nn.Conv2d(64, channels, 3, stride = 1, padding = 1),
                # Can switch to Tanh for LSGAN if needed
                nn.Tanh()
        )

    def forward(self, z):
        out = self.first_layer(z)
        out = out.view(out.shape[0], 128,  self.init_size,  self.init_size)
        img = self.deconvolutional_part(out)
        return img


class Our_Discriminator(nn.Module):
    def __init__(self, batch_norm = True, momentum = 0.8):
        super(Our_Discriminator, self).__init__()

        def conv_block(in_filters, out_filters, kernel_size, stride, padding = 0, bn=batch_norm):
            block = [nn.Conv2d(in_filters, out_filters, kernel_size, stride, padding)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, momentum = momentum))
            block.append(nn.LeakyReLU(0.2, inplace=True))
            return block


        # FORMULA FOR SIZE OF CONVOLUTIONAL AND DECONV LAYERS ARE DIFFERENT
        # We don't have to keep image size same, but do it for convenience of knowing
        # size of last layer
        self.model = nn.Sequential(
            # change from kernel size from 5 to 4 to make padding an integer
            # and keep image size the same
            *conv_block(channels, 16, 3, stride = 2, padding = 1, bn = False),
            *conv_block(16, 32, 3, stride = 2, padding = 1),
            *conv_block(32, 64, 3, stride = 2, padding = 1),
            #*conv_block(64, 128, 4, stride = 2, padding = 15),
            #*conv_block(128, 256, 4, stride = 2, padding = 15),
            #*conv_block(256, 512, 4, stride = 2, padding = 15),
        )

        ds_size = img_size // 2 ** 3
        self.adv_layer = nn.Sequential(
            # Keep  Linear part only for LSGAN
            nn.Linear(64 * ds_size ** 2, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity

In [0]:
class Our_Generator(nn.Module):
    def __init__(self, batch_norm = True, momentum = 0.8):
        super(Our_Generator, self).__init__()
        
        self.init_size = img_size // 16

        def linear_block(in_feat, out_feat, bn=batch_norm):
            layers = [nn.Linear(in_feat, out_feat)]
            if bn:
                layers.append(nn.BatchNorm1d(out_feat, momentum = momentum))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        # as specified in paper
        def deconv_block(input_channels, output_channels, kernel_size, stride, padding, output_padding, bn=batch_norm):
            layers = [nn.ConvTranspose2d(input_channels, output_channels, kernel_size = kernel_size, stride = stride, padding = padding, output_padding = output_padding)]
            if bn:
                layers.append(nn.BatchNorm2d(output_channels, momentum = momentum))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers


        self.first_layer = nn.Sequential(*linear_block(latent_dim, 128 *  self.init_size**2, bn = False))

        # formula to keep 2D images the same size FOR CONVENIENCE
        # https://discuss.pytorch.org/t/how-to-keep-the-shape-of-input-and-output-same-when-dilation-conv/14338/2

        # I used this formula to keep the Image Sizes the same for deconvolutional layers
        # https://www.wolframalpha.com/input/?i=28+%3D+%2828+-+1%29+*+1+-2Z+%2B++P+%2B+%283-1%29+%2B+1

        # FORMULA FOR SIZE OF CONVOLUTIONAL AND DECONV LAYERS ARE DIFFERENT

        self.deconvolutional_part = nn.Sequential(
                *deconv_block(128, 128, kernel_size = 3, stride = 2, padding = 1, output_padding = 1),
                *deconv_block(128, 64, kernel_size = 3, stride = 2, padding = 1, output_padding = 1),
                *deconv_block(64, 32, kernel_size = 3, stride = 2, padding = 1, output_padding = 1),
                *deconv_block(32, 16, kernel_size = 3, stride = 2, padding = 1, output_padding = 1),
                # Taking really long to train
                # Very last deconv part, needs to be right size of image
                # channels is 1 for mnist, 3 for colored datasets
                # NO batch norm here, always
                nn.Conv2d(16, channels, 3, stride = 1, padding = 1),
                # Can switch to Tanh for LSGAN if needed
                nn.Tanh()
        )

    def forward(self, z):
        out = self.first_layer(z)
        out = out.view(out.shape[0], 128,  self.init_size,  self.init_size)
        img = self.deconvolutional_part(out)
        return img

In [0]:
#gan_no_batch = Our_Generator(batch_norm = False)
#lsgan_no_batch = Our_Generator(batch_norm = False)
#gan_batch = Our_Generator(batch_norm = True, momentum = 0.05)
lsgan_batch = Our_Generator(batch_norm = True, momentum = 0.1)

In [0]:
gan_no_batch.load_state_dict(torch.load(BASE_DIR + "/data/our_gan/no-batch-fashion-mnist/generator.pt"))
gan_no_batch.eval()
gan_no_batch.cuda()

Our_Generator(
  (first_layer): Sequential(
    (0): Linear(in_features=1024, out_features=8192, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (deconvolutional_part): Sequential(
    (0): ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): Tanh()
  )
)

In [0]:
gan_batch.load_state_dict(torch.load(BASE_DIR + "/data/our_gan/momentum.05-batch-fashion-mnist/generator.pt"))
gan_batch.eval()
gan_batch.cuda()

Our_Generator(
  (first_layer): Sequential(
    (0): Linear(in_features=1024, out_features=8192, bias=True)
    (1): BatchNorm1d(8192, eps=1e-05, momentum=0.05, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (deconvolutional_part): Sequential(
    (0): ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.05, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.05, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
    (6): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): Tanh()
  )
)

In [0]:
lsgan_no_batch.load_state_dict(torch.load(BASE_DIR + "/data/lsgan/no-batch-fashion-mnist/generator_momentum.pt"))
lsgan_no_batch.eval()
lsgan_no_batch.cuda()

Our_Generator(
  (first_layer): Sequential(
    (0): Linear(in_features=1024, out_features=8192, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (deconvolutional_part): Sequential(
    (0): ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): Tanh()
  )
)

In [0]:
#lsgan_batch.load_state_dict(torch.load(BASE_DIR + "/data/lsgan/momentum-0.1-batch-fashion-mnist/generator_momentum.pt"))
#lsgan_batch.eval()
#lsgan_batch.cuda()

Our_Generator(
  (first_layer): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (deconvolutional_part): Sequential(
    (0): ConvTranspose2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
    (6): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): ConvTranspose2d(32, 16, kernel_size=(3

In [0]:
lsgan_batch.load_state_dict(torch.load(BASE_DIR + "/data/lsgan/batch-fashion-mnist/generator_momentum.pt"))
lsgan_batch.eval()
lsgan_batch.cuda()

# Inception Model Class

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

try:
    from torchvision.models.utils import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url

# Inception weights ported to Pytorch from
# http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz
FID_WEIGHTS_URL = 'https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth'


class InceptionV3(nn.Module):
    """Pretrained InceptionV3 network returning feature maps"""

    # Index of default block of inception to return,
    # corresponds to output of final average pooling
    DEFAULT_BLOCK_INDEX = 3

    # Maps feature dimensionality to their output blocks indices
    BLOCK_INDEX_BY_DIM = {
        64: 0,   # First max pooling features
        192: 1,  # Second max pooling featurs
        768: 2,  # Pre-aux classifier features
        2048: 3  # Final average pooling features
    }

    def __init__(self,
                 output_blocks=[DEFAULT_BLOCK_INDEX],
                 resize_input=True,
                 normalize_input=True,
                 requires_grad=False,
                 use_fid_inception=True):
        """Build pretrained InceptionV3
        Parameters
        ----------
        output_blocks : list of int
            Indices of blocks to return features of. Possible values are:
                - 0: corresponds to output of first max pooling
                - 1: corresponds to output of second max pooling
                - 2: corresponds to output which is fed to aux classifier
                - 3: corresponds to output of final average pooling
        resize_input : bool
            If true, bilinearly resizes input to width and height 299 before
            feeding input to model. As the network without fully connected
            layers is fully convolutional, it should be able to handle inputs
            of arbitrary size, so resizing might not be strictly needed
        normalize_input : bool
            If true, scales the input from range (0, 1) to the range the
            pretrained Inception network expects, namely (-1, 1)
        requires_grad : bool
            If true, parameters of the model require gradients. Possibly useful
            for finetuning the network
        use_fid_inception : bool
            If true, uses the pretrained Inception model used in Tensorflow's
            FID implementation. If false, uses the pretrained Inception model
            available in torchvision. The FID Inception model has different
            weights and a slightly different structure from torchvision's
            Inception model. If you want to compute FID scores, you are
            strongly advised to set this parameter to true to get comparable
            results.
        """
        super(InceptionV3, self).__init__()

        self.resize_input = resize_input
        self.normalize_input = normalize_input
        self.output_blocks = sorted(output_blocks)
        self.last_needed_block = max(output_blocks)

        assert self.last_needed_block <= 3, \
            'Last possible output block index is 3'

        self.blocks = nn.ModuleList()

        if use_fid_inception:
            inception = fid_inception_v3()
        else:
            inception = models.inception_v3(pretrained=True)

        # Block 0: input to maxpool1
        block0 = [
            inception.Conv2d_1a_3x3,
            inception.Conv2d_2a_3x3,
            inception.Conv2d_2b_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2)
        ]
        self.blocks.append(nn.Sequential(*block0))

        # Block 1: maxpool1 to maxpool2
        if self.last_needed_block >= 1:
            block1 = [
                inception.Conv2d_3b_1x1,
                inception.Conv2d_4a_3x3,
                nn.MaxPool2d(kernel_size=3, stride=2)
            ]
            self.blocks.append(nn.Sequential(*block1))

        # Block 2: maxpool2 to aux classifier
        if self.last_needed_block >= 2:
            block2 = [
                inception.Mixed_5b,
                inception.Mixed_5c,
                inception.Mixed_5d,
                inception.Mixed_6a,
                inception.Mixed_6b,
                inception.Mixed_6c,
                inception.Mixed_6d,
                inception.Mixed_6e,
            ]
            self.blocks.append(nn.Sequential(*block2))

        # Block 3: aux classifier to final avgpool
        if self.last_needed_block >= 3:
            block3 = [
                inception.Mixed_7a,
                inception.Mixed_7b,
                inception.Mixed_7c,
                nn.AdaptiveAvgPool2d(output_size=(1, 1))
            ]
            self.blocks.append(nn.Sequential(*block3))

        for param in self.parameters():
            param.requires_grad = requires_grad

    def forward(self, inp):
        """Get Inception feature maps
        Parameters
        ----------
        inp : torch.autograd.Variable
            Input tensor of shape Bx3xHxW. Values are expected to be in
            range (0, 1)
        Returns
        -------
        List of torch.autograd.Variable, corresponding to the selected output
        block, sorted ascending by index
        """
        outp = []
        x = inp

        if self.resize_input:
            x = F.interpolate(x,
                              size=(299, 299),
                              mode='bilinear',
                              align_corners=False)

        if self.normalize_input:
            x = 2 * x - 1  # Scale from range (0, 1) to range (-1, 1)

        for idx, block in enumerate(self.blocks):
            x = block(x)
            if idx in self.output_blocks:
                outp.append(x)

            if idx == self.last_needed_block:
                break

        return outp


def fid_inception_v3():
    """Build pretrained Inception model for FID computation
    The Inception model for FID computation uses a different set of weights
    and has a slightly different structure than torchvision's Inception.
    This method first constructs torchvision's Inception and then patches the
    necessary parts that are different in the FID Inception model.
    """
    inception = models.inception_v3(num_classes=1008,
                                    aux_logits=False,
                                    pretrained=False)
    inception.Mixed_5b = FIDInceptionA(192, pool_features=32)
    inception.Mixed_5c = FIDInceptionA(256, pool_features=64)
    inception.Mixed_5d = FIDInceptionA(288, pool_features=64)
    inception.Mixed_6b = FIDInceptionC(768, channels_7x7=128)
    inception.Mixed_6c = FIDInceptionC(768, channels_7x7=160)
    inception.Mixed_6d = FIDInceptionC(768, channels_7x7=160)
    inception.Mixed_6e = FIDInceptionC(768, channels_7x7=192)
    inception.Mixed_7b = FIDInceptionE_1(1280)
    inception.Mixed_7c = FIDInceptionE_2(2048)

    state_dict = load_state_dict_from_url(FID_WEIGHTS_URL, progress=True)
    inception.load_state_dict(state_dict)
    return inception


class FIDInceptionA(models.inception.InceptionA):
    """InceptionA block patched for FID computation"""
    def __init__(self, in_channels, pool_features):
        super(FIDInceptionA, self).__init__(in_channels, pool_features)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        # Patch: Tensorflow's average pool does not use the padded zero's in
        # its average calculation
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1,
                                   count_include_pad=False)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)


class FIDInceptionC(models.inception.InceptionC):
    """InceptionC block patched for FID computation"""
    def __init__(self, in_channels, channels_7x7):
        super(FIDInceptionC, self).__init__(in_channels, channels_7x7)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch7x7 = self.branch7x7_1(x)
        branch7x7 = self.branch7x7_2(branch7x7)
        branch7x7 = self.branch7x7_3(branch7x7)

        branch7x7dbl = self.branch7x7dbl_1(x)
        branch7x7dbl = self.branch7x7dbl_2(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_3(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_4(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_5(branch7x7dbl)

        # Patch: Tensorflow's average pool does not use the padded zero's in
        # its average calculation
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1,
                                   count_include_pad=False)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch7x7, branch7x7dbl, branch_pool]
        return torch.cat(outputs, 1)


class FIDInceptionE_1(models.inception.InceptionE):
    """First InceptionE block patched for FID computation"""
    def __init__(self, in_channels):
        super(FIDInceptionE_1, self).__init__(in_channels)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch3x3 = self.branch3x3_1(x)
        branch3x3 = [
            self.branch3x3_2a(branch3x3),
            self.branch3x3_2b(branch3x3),
        ]
        branch3x3 = torch.cat(branch3x3, 1)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = [
            self.branch3x3dbl_3a(branch3x3dbl),
            self.branch3x3dbl_3b(branch3x3dbl),
        ]
        branch3x3dbl = torch.cat(branch3x3dbl, 1)

        # Patch: Tensorflow's average pool does not use the padded zero's in
        # its average calculation
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1,
                                   count_include_pad=False)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch3x3, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)


class FIDInceptionE_2(models.inception.InceptionE):
    """Second InceptionE block patched for FID computation"""
    def __init__(self, in_channels):
        super(FIDInceptionE_2, self).__init__(in_channels)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch3x3 = self.branch3x3_1(x)
        branch3x3 = [
            self.branch3x3_2a(branch3x3),
            self.branch3x3_2b(branch3x3),
        ]
        branch3x3 = torch.cat(branch3x3, 1)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = [
            self.branch3x3dbl_3a(branch3x3dbl),
            self.branch3x3dbl_3b(branch3x3dbl),
        ]
        branch3x3dbl = torch.cat(branch3x3dbl, 1)

        # Patch: The FID Inception model uses max pooling instead of average
        # pooling. This is likely an error in this specific Inception
        # implementation, as other Inception models use average pooling here
        # (which matches the description in the paper).
        branch_pool = F.max_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch3x3, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

# Initialize Inception Model

The code for it is inside the code above

# Generate images for the different generators

In [0]:
n_batches = count_test // batch_size

In [0]:
n_batches

200

In [0]:
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [0]:
z = Variable(Tensor(np.random.normal(0, 1, (batch_size, latent_dim))))

In [0]:
def gen_test_images(generator):
    gn = [generator(Variable(Tensor(np.random.normal(0, 1, (batch_size, latent_dim))))).cpu().detach().numpy() for i in range(n_batches)]
    return np.concatenate(gn)

In [0]:
#gen_gan_no_batch_imgs = gen_test_images(gan_no_batch)
#gen_lsgan_no_batch_imgs = gen_test_images(lsgan_no_batch)

#gen_gan_batch_imgs = gen_test_images(gan_batch)
gen_lsgan_batch_imgs = gen_test_images(lsgan_batch)

In [0]:
real_imgs = []
for i, (imgs, _) in enumerate(fashion_dataloader_test):
    real_imgs.append(imgs.cpu().detach().numpy())
real_imgs = np.concatenate(real_imgs)

In [0]:
real_imgs.shape

(10000, 1, 32, 32)

Inception requires 3 channels. So just replicate 3 times.

In [0]:
def rep_channel(data):
    return np.concatenate((data, data, data), axis = 1)

In [0]:
#gen_gan_no_batch_imgs = rep_channel(gen_gan_no_batch_imgs)
#gen_lsgan_no_batch_imgs = rep_channel(gen_lsgan_no_batch_imgs)
#gen_gan_batch_imgs = rep_channel(gen_gan_batch_imgs)
gen_lsgan_batch_imgs = rep_channel(gen_lsgan_batch_imgs)

real_imgs = rep_channel(real_imgs)

# Calculate scores for the different generators

In [0]:
def tqdm(x): return x

In [0]:
fid_value_gan_no_batch = calculate_fid_given_paths([gen_gan_no_batch_imgs, real_imgs],
                                          batch_size,
                                          cuda = cuda,
                                          dims = dims)

print("GAN NO BATCH")
print(fid_value_gan_no_batch)

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/checkpoints/pt_inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:02<00:00, 38.0MB/s]


GAN NO BATCH
66.55188455053613


In [0]:
fid_value_gan_batch = calculate_fid_given_paths([gen_gan_batch_imgs, real_imgs],
                                          batch_size,
                                          cuda = cuda,
                                          dims = dims)

print("GAN BATCH")
print(fid_value_gan_batch)

GAN BATCH
348.0485219296422


In [0]:
fid_value_lsgan_no_batch = calculate_fid_given_paths([gen_lsgan_no_batch_imgs, real_imgs],
                                          batch_size,
                                          cuda = cuda,
                                          dims = dims)

print("LSGAN NO BATCH")
print(fid_value_lsgan_no_batch)

LSGAN NO BATCH
48.66737531511262


In [0]:
fid_value_lsgan_batch = calculate_fid_given_paths([gen_lsgan_batch_imgs, real_imgs],
                                          batch_size,
                                          cuda = cuda,
                                          dims = dims)

print("LSGAN BATCH")
print(fid_value_lsgan_batch)

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/checkpoints/pt_inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:03<00:00, 29.7MB/s]


LSGAN BATCH
94.17686727454196
